In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from scipy.spatial.transform import Rotation 

In [2]:
# functions 
# TODO: make a utils file and import these functions  

def pose_to_T(pose):  
    if len(pose) == 6: 
        # x,y,z,a,b,c 
        position = pose[:3].reshape(3,1) 
        a = pose[3] 
        b = pose[4]
        c = pose[5] 
        rotation = Rotation.from_euler('xyz',[c,b,a],degrees=True).as_matrix() 
    elif len(pose) == 7: 
        # x,y,z,qx,qy,qz,qw 
        position = pose[:3].reshape(3,1) 
        q = pose[3:7] # scalar last convention 
        rotation = Rotation.from_quat(q).as_matrix() 
    else: 
        print('Invalid pose dimension') 
        return 
    T = np.vstack((
        np.hstack((rotation,position)),
        np.array([0,0,0,1]) 
    )) 
    return T 

def T_to_pose(T, pose_dim=7):
    x, y, z = T[0,3], T[1,3], T[2,3] 
    if pose_dim == 6: 
        c, b, a = Rotation.from_matrix(T[:3,:3]).as_euler('xyz',degrees=True)     
        pose = np.array([x,y,z,a,b,c])  
    elif pose_dim == 7: 
        q = Rotation.from_matrix(T[:3,:3]).as_quat() 
        pose = np.zeros(7) 
        pose[:3] = np.array([x,y,z]) 
        pose[3:] = q 
    return pose 

def pose_6D_to_7D(pose_6D): 
    pose_7D = np.empty(7) 
    pose_7D[:3] = pose_6D[:3]
    pose_7D[3:] = Rotation.from_euler('xyz', pose_6D[3:], degrees=True).as_quat() 
    return pose_7D 


def pose_7D_to_6D(pose_7D): 
    pose_6D = np.empty(6) 
    pose_6D[:3] = pose_7D[:3] 
    C, B, A = Rotation.from_quat(pose_7D[3:]).as_euler('xyz', degrees=True)  
    pose_6D[3:] = np.array([A, B, C]) 
    return pose_6D 

def apply_delta_pose(pose, delta_pose): 
    # note delta pose is with respect to the local frame A 
    # e.g. if pose is the robot pose, then delta pose is with respect to tool frame 
    T_0A = pose_to_T(pose) 
    T_AB = pose_to_T(delta_pose) 
    T_0B = T_0A @ T_AB 
    pose_new = T_to_pose(T_0B) 
    return pose_new 

def invert_pose(pose, pose_dim=7): 
    T = T_to_pose(pose) 
    T_inv = np.linalg.inv(T) 
    pose_inv = T_to_pose(T_inv, pose_dim=pose_dim)  
    return pose_inv 

In [3]:
# read in data 
trial_num = 4  
dir_data = "/media/rp/Elements/abhay_ws/real_contact_data/" 

df_pose = pd.read_csv(dir_data + "pose.csv")
df_wrench = pd.read_csv(dir_data + "wrench.csv") 

print(df_pose.columns)
print(df_wrench.columns) 

# process time for each df  
df_pose['t'] = df_pose['sec'] + 1e-9*df_pose['nanosec']
df_wrench['t'] = df_wrench['sec'] + 1e-9*df_wrench['nanosec'] 

# check which df has higher sampling rate and use that as master time to merge the two df 
mean_dt_pose = np.mean(df_pose['t'].diff()) 
mean_dt_wrench = np.mean(df_wrench['t'].diff()) 

# slower data rate leads 
if mean_dt_pose < mean_dt_wrench: 
    # pose saved at higher rate than wrench 
    df_time_lead = df_wrench 
    df_time_follow = df_pose 
else: 
    # wrench saved at higher rate than pose 
    df_time_lead = df_pose  
    df_time_follow = df_wrench  

t_lead = df_time_lead['t'].values
t_follow = df_time_follow['t'].values   

df_data = df_time_lead 
columns_add = df_time_follow.columns.to_list() 
columns_add.remove('t')
columns_add.remove('sec')
columns_add.remove('nanosec') 

df_data[columns_add] = 0 
# interpolate data 
for i, ti_L in enumerate(t_lead): 
    for col in columns_add: 
        df_data.loc[i, col] = np.interp(ti_L, t_follow, df_time_follow[col].values) 

print(df_data.columns) 
print(len(df_data))

Index(['sec', 'nanosec', 'X', 'Y', 'Z', 'qx', 'qy', 'qz', 'qw'], dtype='object')
Index(['sec', 'nanosec', 'FX', 'FY', 'FZ', 'TX', 'TY', 'TZ'], dtype='object')


/tmp/ipykernel_534636/2027146135.py:42: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.5676898281909104' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_data.loc[i, col] = np.interp(ti_L, t_follow, df_time_follow[col].values)
/tmp/ipykernel_534636/2027146135.py:42: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.8997774183629977' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_data.loc[i, col] = np.interp(ti_L, t_follow, df_time_follow[col].values)
/tmp/ipykernel_534636/2027146135.py:42: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-2.5078868486526225' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_data.loc[i

Index(['sec', 'nanosec', 'X', 'Y', 'Z', 'qx', 'qy', 'qz', 'qw', 't', 'FX',
       'FY', 'FZ', 'TX', 'TY', 'TZ'],
      dtype='object')
182520


In [4]:
# transform to tool frame 
pose_tool_wrt_flange = np.array([0,0,+110.0e-3,0,1,0,0]) # peg tip 
pose_insert = np.array([694.93e-3, 88.11e-3, 195.51e-3, -178.99, 0.56, -179.52]) 

# frame definitions: 
# B: base (robot) 
# F: flange (+z is direction of insertion) 
# T: tool (peg) (-z is direction of insertion) 
# Ts: tool (peg) at success (full insertion) 
# T0: tool (peg) at start (no insertion), origin of tip at same point as top surface of hole 

T_FT = pose_to_T(pose_tool_wrt_flange) 
T_TF = np.linalg.inv(T_FT)
T_BFs = pose_to_T(pose_insert) 
T_FsB = np.linalg.inv(T_BFs)
T_TsB = T_TF @ T_FsB 
T_TsT0 = pose_to_T(np.array([0,0,0,0,0,0,1])) 
T_T0Ts = np.linalg.inv(T_TsT0) 
T_T0B = T_T0Ts @ T_TsB 

df_tool = pd.DataFrame() 
df_tool[['t','X','Y','Z','A','B','C']] = 0 
df_tool['t'] = df_data['t'] - df_data.loc[0,'t'] 

for i, row in df_data.iterrows(): 
    T_BFi = pose_to_T(row[['X','Y','Z','qx','qy','qz','qw']].values) 
    T_T0Ti = T_T0B @ T_BFi @ T_FT 
    df_tool.loc[i,['X','Y','Z','A','B','C']] = T_to_pose(T_T0Ti, pose_dim=6) 

    # print progress every 10% 
    if i % (len(df_data)//10) == 0: 
        print(f'Progress: {i}/{len(df_data)}') 

df_tool[['X','Y','Z']] *= 1E3 

# add wrench data to df_tool 
dim_wrench = ['FX','FY','FZ','TX','TY','TZ']
df_tool[dim_wrench] = df_data[dim_wrench] 

Progress: 0/182520
Progress: 18252/182520
Progress: 36504/182520
Progress: 54756/182520
Progress: 73008/182520
Progress: 91260/182520
Progress: 109512/182520
Progress: 127764/182520
Progress: 146016/182520
Progress: 164268/182520


In [10]:
# two subplots of Z and FZ 
%matplotlib qt 
fig, ax = plt.subplots(2,1, figsize=(12,12), sharex=True)
ax[0].plot(df_tool['t'], df_tool['Z'], label='Z')
ax[0].set_title('Z')
ax[0].set_xlabel('Time (s)')
ax[0].set_ylabel('Z (mm)')
ax[0].grid(True)

ax[1].plot(df_tool['t'], df_tool['FZ'], label='FZ')
ax[1].set_title('FZ')
ax[1].set_xlabel('Time (s)')
ax[1].set_ylabel('FZ (N)')
ax[1].grid(True)

plt.show() 


In [6]:
# identify contact points 
fz_contact_thresh = -3.0 
Fxy_contact_thresh = 10.0 
df_tool['contact'] = False 
for i, row in df_tool.iterrows(): 
    if row['FZ'] < fz_contact_thresh: 
        df_tool.loc[i,'contact'] = True 
    elif np.abs(row['FX']) > Fxy_contact_thresh: 
        df_tool.loc[i,'contact'] = True 
    elif np.abs(row['FY']) > Fxy_contact_thresh: 
        df_tool.loc[i,'contact'] = True 
    else: 
        df_tool.loc[i,'contact'] = False 

# print number of points labeled as contact and total number of points 
print(np.sum([df_tool['contact'] == True]))
print(len(df_tool)) 

123461
182520


In [7]:
# save data 
df_tool.to_csv(dir_data + f"cross_peg_contact_mapping_real.csv", index=False) 